In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
import os 
from glob import glob
from scipy.io import loadmat
from tqdm import tqdm

In [6]:
def intersection(line1, line2):
    # Cartesian coordinates (x1, y1, x2, y2)
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Calculate the determinant
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    
    if denom == 0:
        return None  # Lines are parallel or coincident
    
    # Calculate the intersection point
    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
    
    return (int(intersect_x), int(intersect_y))

In [7]:
def filter_nearby_intersections(intersections, threshold=10):
    """
    Filters out intersections that are within the given threshold distance.
    
    Args:
    - intersections: List of intersection points (x, y).
    - threshold: The distance below which intersections are considered duplicates (default 10 pixels).
    
    Returns:
    - A list of unique intersection points.
    """
    filtered = []
    
    for point in intersections:
        add_point = True
        for filtered_point in filtered:
            # Calculate Euclidean distance between the current point and each filtered point
            dist = np.sqrt((point[0] - filtered_point[0]) ** 2 + (point[1] - filtered_point[1]) ** 2)
            if dist < threshold:
                add_point = False  # Don't add this point, it's too close to an existing one
                break
        if add_point:
            filtered.append(point)
    
    return filtered

In [12]:
def perspective_transform(image, corners, im_size=900):
    # Define the width and height for the output square
    width, height = im_size, im_size
    
    # Define the destination points (square corners in the destination image)
    dst_points = np.array([
        [0, 0],  # top-left
        [width - 1, 0],  # top-right
        [width - 1, height - 1],  # bottom-right
        [0, height - 1]  # bottom-left
    ], dtype="float32")
    
    # If 4 corners are detected, proceed
    if len(corners) == 4:
        # Sort the corners in a consistent order
        sorted_corners = sorted(corners, key=lambda x: (x[1], x[0]))  # Sort by y-coordinate first, then by x-coordinate
        
        # Now that we have the sorted corners, we need to assign them to top-left, top-right, bottom-left, bottom-right
        (tl, tr, br, bl) = sorted_corners
        
        # Further classify based on x-coordinate (top-left vs top-right and bottom-left vs bottom-right)
        if tl[0] > tr[0]:
            tl, tr = tr, tl
        if bl[0] > br[0]:
            bl, br = br, bl
        
        # Define the source points from the detected corners
        src_points = np.array([tl, tr, br, bl], dtype="float32")
        
        # Compute the perspective transform matrix
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        
        # Apply the perspective transformation
        transformed_image = cv2.warpPerspective(image, M, (width, height))
        
        return transformed_image
    else:
        return None

In [8]:
def calculate_angle(line1, line2):
    # Extract coordinates from the lines
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Calculate direction vectors
    vector1 = np.array([x2 - x1, y2 - y1])
    vector2 = np.array([x4 - x3, y4 - y3])
    
    # Compute dot product and magnitudes
    dot_product = np.dot(vector1, vector2)
    mag1 = np.linalg.norm(vector1)
    mag2 = np.linalg.norm(vector2)
    
    # Calculate the cosine of the angle
    cos_theta = dot_product / (mag1 * mag2)
    
    # Ensure cos_theta is within valid range for acos to avoid errors due to floating point precision
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    
    # Calculate the angle in radians
    angle_rad = np.arccos(cos_theta)
    
    # Convert the angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

In [9]:
def find_bounding_rectangle(blob):
    # find 4 longest lines
    edges = cv2.Canny(blob, 150, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=100)
    line_details = []
    
    # Calculate the length of each line and store the details
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            line_details.append((x1, y1, x2, y2, length))
    
    # Sort the lines by length in descending order
    line_details.sort(key=lambda x: x[4], reverse=True)
    
    # Load the original image for drawing the lines
    viz_im = im_rgb.copy()
    
    # Draw the 4 longest lines on the image
    for i in range(min(10, len(line_details))):
        x1, y1, x2, y2, length = line_details[i]
        cv2.line(viz_im, (x1, y1), (x2, y2), (255, 0, 0), 6)  # Draw the line in gree

    intersections = []
    if lines is not None:
        for i in range(min(10, len(line_details))):
            for j in range(min(10, len(line_details))):
                line1 = line_details[i][0:4]
                line2 = line_details[j][0:4]
                if calculate_angle(line1, line2) < 45:
                    continue
                intersect_point = intersection(line1, line2)
                if intersect_point and 0 <= intersect_point[0] <= blob.shape[1] and 0 <= intersect_point[1] < blob.shape[0]:
                    intersections.append(intersect_point)

    filtered_intersectioins = filter_nearby_intersections(intersections, 30)
    
    for point in filtered_intersectioins:
        cv2.circle(viz_im, point, 10, (0, 255, 0), -1)  # Draw intersections as green circles

    return viz_im, filtered_intersectioins

## Segment the screw holder

In [5]:
im_parent_dir = "captured_screw_images"
output_im_dir = "captured_screw_datasets"
os.makedirs(output_im_dir, exist_ok=True)

train_dirs = ["pose01", "pose02", "pose03", "pose04", "pose05", "pose05_color01"]
valid_dirs = ["pose06", "pose06_color01"]
test_dirs = ["pose07", "pose07_color01"]

im_fps = glob(os.path.join(im_parent_dir, "*/*bmp"))
len(im_fps)

61

In [23]:
viz = False
for fp in tqdm(im_fps):
    im_bgr = cv2.imread(fp)
    im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_RGB2BGR)
    if viz:
        plt.imshow(im_rgb)
        plt.show()
    
    im_g = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)
    im_bi = np.zeros_like(im_g)
    # im_bi[im_g >= 160] = 255
    im_bi[im_g >= 125] = 255
    if viz:
        plt.imshow(im_bi, cmap="gray")
        plt.show()
    
    # get biggest blob
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(im_bi)
    largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])  # Skip the background (label 0)
    largest_component = (labels == largest_label).astype(np.uint8) * 255
    
    # fill screw holes
    # des = cv2.bitwise_not(largest_component)
    cnts, hier = cv2.findContours(largest_component, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        cv2.drawContours(largest_component,[cnt],0,255,-1)
    
    contours, _ = cv2.findContours(largest_component, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Loop through each contour to fit a polygon
    contour = contours[0] # get the first and only contour
    # Approximate the polygon with a precision proportional to the perimeter
    hull = cv2.convexHull(contour)
    cv2.fillPoly(largest_component, [hull], color=255)

    if viz:
        plt.imshow(largest_component, cmap="gray")
        plt.show()
    
    viz_im, corners = find_bounding_rectangle(largest_component)
    
    if len(corners) == 4:
        screw_im = perspective_transform(im_rgb, corners, im_size=416*3)
        if screw_im is not None and viz:
            cv2.line(screw_im, (300, 0), (300, 900), (255, 0, 0), 3)
            cv2.line(screw_im, (600, 0), (600, 900), (255, 0, 0), 3)
            cv2.line(screw_im, (0, 300), (900, 300), (255, 0, 0), 3)
            cv2.line(screw_im, (0, 600), (900, 600), (255, 0, 0), 3)
            plt.imshow(screw_im)
            plt.show()

        if screw_im is not None:
            im_dir = os.path.dirname(fp).split("\\")[-1]
            if im_dir in train_dirs:
                out_dir = os.path.join(output_im_dir, "train")
            elif im_dir in valid_dirs:
                out_dir = os.path.join(output_im_dir, "validation")
            else:
                out_dir = os.path.join(output_im_dir, "test")
            os.makedirs(out_dir, exist_ok=True)
            for r in range(3):
                for c in range(3):
                    crop_im = screw_im[r*416:(r+1)*416, c*416:(c+1)*416]
                    cv2.imwrite(os.path.join(out_dir, f"{im_dir}_{os.path.basename(fp).split('.')[0]}_{r+1}_{c+1}.jpg"), cv2.cvtColor(crop_im, cv2.COLOR_RGB2BGR))
    else:
        print(f"[ERROR] Got an error on the image at: {fp}")
        
    if viz:   
        plt.imshow(viz_im)

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:04<00:00, 13.18it/s]
